In [4]:
import pandas as pd
import os
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import gc
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import seaborn as sns
import tensorflow as tf
import warnings
import re

In [7]:
tf.debugging.set_log_device_placement(True)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU') 
        #tf.config.experimental.set_memory_growth(gpus[0], True)
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
else:
    print('No GPU detected')   

No GPU detected


In [6]:
print(tf.__version__)

2.3.0


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow import keras
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam

In [5]:
tf.__version__

'2.3.1'

In [6]:
ls "."

 C 드라이브의 볼륨: SSD
 볼륨 일련 번호: BCA4-98AF

 C:\Users\inhoinno\DACON\2020_DACON-Kaggle\소설저자AI 디렉터리

2020-11-26  오후 06:15    <DIR>          .
2020-11-26  오후 06:15    <DIR>          ..
2020-11-25  오전 12:07    <DIR>          .ipynb_checkpoints
2020-11-24  오전 10:49            33,874 Augmentation.ipynb
2020-11-24  오전 11:15    <DIR>          dataset
2020-11-05  오후 05:31    <DIR>          ensemble
2020-10-09  오후 05:05         1,327,724 Logistic_regression.ipynb
2020-10-29  오후 06:12         3,959,848 NPL_base_5layer.h5
2020-11-12  오전 11:23            35,818 NPL_BaseLine.ipynb
2020-11-21  오후 11:47           992,363 NPL_BaseLine-CV.ipynb
2020-11-21  오후 11:47           992,363 NPL_BaseLine-CV-Copy1.ipynb
2020-11-05  오전 12:07           934,892 NPL_BaseLine-LSTM.ipynb
2020-11-12  오전 11:23           942,567 NPL_BaseLine-ONE-HOT.ipynb
2020-11-20  오후 05:49           115,342 NPL_Base-Tfidf-CV.ipynb
2020-11-20  오후 05:49           115,342 NPL_Base-Tfidf-CV-Copy1.ipynb
2020-11-12  오전 11:23           927,110 NP

In [7]:
#파일 불러오기
train = pd.read_csv('open/train_processed_500.csv', encoding = 'utf-8')
train_origin = pd.read_csv('open/train.csv', encoding = 'utf-8')
test = pd.read_csv('open/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('open/sample_submission.csv', encoding = 'utf-8')

In [8]:
train.describe()

,index,author,length
count,183468.000000,183468.000000,183468.000000
mean,91733.500000,2.049088,162.545567
std,52962.793931,1.285455,178.318420
min,0.000000,0.000000,4.000000
25%,45866.750000,1.000000,63.000000
50%,91733.500000,2.000000,112.000000
75%,137600.250000,3.000000,188.000000
max,183467.000000,4.000000,2454.000000


In [9]:
train['length'] = train['text'].apply(len)

In [10]:
train[train['length'] < 20]

,index,text,author,length
107,107,Tell me what it is.,0,19
219,219,I said Not at all.,0,18
418,418,I dare say she had.,1,19
829,829,I am to be neutral,2,18
873,873,And where was it,2,17
...,...,...,...,...
182119,182119,Yes said he,3,13
182148,182148,He thus continued,1,18
182210,182210,Wait up for me,1,15
182241,182241,How do you do Mr,1,18


<h2>텍스트 전처리

In [11]:

#부호를 제거해주는 함수

def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)


#1. 불용어 제거 remove_stopwords
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [12]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num)#.apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num)#.apply(remove_stopwords)

In [13]:
train[train['length'] < 20]

,index,text,author,length
107,107,tell me what it is,0,19
219,219,i said not at all,0,18
418,418,i dare say she had,1,19
829,829,i am to be neutral,2,18
873,873,and where was it,2,17
...,...,...,...,...
182119,182119,yes said he,3,13
182148,182148,he thus continued,1,18
182210,182210,wait up for me,1,15
182241,182241,how do you do mr,1,18


In [14]:
idx_under_20 = train[train['length'] < 20].index
train = train.drop(idx_under_20)
train.describe()

,index,author,length
count,181013.000000,181013.000000,181013.000000
mean,91514.586997,2.049146,164.537862
std,53025.418222,1.286300,178.695183
min,0.000000,0.000000,20.000000
25%,45490.000000,1.000000,65.000000
50%,91441.000000,2.000000,113.000000
75%,137458.000000,3.000000,189.000000
max,183467.000000,4.000000,2454.000000


In [15]:

train

,index,text,author,length
0,0,he was almost choking there was so much so muc...,3,235
1,1,your sister asked for it i suppose,2,34
2,2,she was engaged one day as she walked in peru...,1,312
3,3,the captain was in the porch keeping himself c...,4,305
4,4,have mercy gentlemen odin flung up his hands d...,3,215
...,...,...,...,...
183463,183463,to be urging her opinion against sir odins wa...,1,571
183464,183464,after many tears an unwritten contract was dr...,3,536
183465,183465,they express the consciousness that you have ...,3,525
183466,183466,and of course he knows himself that he is doi...,3,515


<h2> 1. 토큰화

One-hot, 해싱, Bow(< 이건 RF와 LGBM에서 사용하자)

[파이썬 라이브러리를 활용한 머신러닝] 7.3장 : 텍스트 데이터를 BOW로 표현하기 

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np 

In [17]:
maxlen = 1000 #100개 이후 단어는 버려
max_words = 20000 #데이터셋에서 가장 빈도 높은 1만개의 단어만 사용

In [18]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['text'])
sequences = tokenizer.texts_to_sequences(train['text']) 

In [19]:
word_index = tokenizer.word_index
print('%s개의 고유한 토큰 확인'%len(word_index))

47272개의 고유한 토큰 확인


In [20]:
data = pad_sequences(sequences, maxlen= maxlen)
print('데이터 텐서 크기',data.shape)

데이터 텐서 크기 (181013, 1000)


In [21]:
y_train = train['author']

<h2> 2. 벡터화 (임베딩)

In [22]:
glove_dir = './dataset/glove.6B/'
embeddings_index={}
f = open(os.path.join(glove_dir,'glove.6B.50d.txt'), encoding ="utf8")

for line in f : 
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word]=coefs
f.close()

In [23]:
print('%s개의 단어 벡터 확인 '%len(embeddings_index))

400001개의 단어 벡터 확인 


In [24]:
word_index = tokenizer.word_index

In [25]:
#Glove 단어 임베딩 행렬 준비하기
#임베딩 전처리
#임베딩 행렬 준비하기
embedding_dim=50
#global embedding_matrix
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i< max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None :
            embedding_matrix[i] = embedding_vector# 임베딩 인덱스에 없는 단어는 모두 0이 됨.

<h2> 모델 정의하기

In [26]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense,GlobalAveragePooling2D
from keras.layers import LSTM, Activation,Dropout

<h3> 모델 하이퍼 파라미터 설정

In [27]:
model_name = 'rnn_classifier_en'
BATCH_SIZE = 128
NUM_EPOCHS = 5
#VALID_SPLIT = 0.1
MAX_LEN = 1000

kargs = {
    'model_name':model_name,
    'vocab_size':max_words,
    'embedding_vector':embedding_matrix,
    'embedding_dimension':100,
    'dropout_rate':0.2,
    'lstm_dimension':32,
    'dense_dimension':32,
    'output_dimension':5
    
}

In [28]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.41800001,  0.24968   , -0.41242   , ..., -0.18411   ,
        -0.11514   , -0.78580999],
       [ 0.26818001,  0.14346001, -0.27877   , ..., -0.63209999,
        -0.25027999, -0.38097   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.89973003, -0.46588001, -0.21157999, ...,  1.00670004,
         0.0065529 ,  0.57164001],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

<h3> 모델 설계

In [29]:
class RNNClassifier(tf.keras.Model):
    def __init__(self,**kargs):
        super(RNNClassifier,self).__init__(name=kargs['model_name'])
        self.embedding = tf.keras.layers.Embedding(input_dim= kargs['vocab_size'],
                                                   weights = [embedding_matrix],
                                                   trainable = False,
                                         output_dim=kargs['embedding_dimension'])
        self.lstm_1_layer = tf.keras.layers.LSTM(kargs['lstm_dimension'],
                                                return_sequences = True)
        self.lstm_2_layer = tf.keras.layers.LSTM(kargs['lstm_dimension'])
        self.dropout = tf.keras.layers.Dropout(kargs['dropout_rate'])
        self.fc1 = tf.keras.layers.Dense(units=kargs['dense_dimension'],
                               activation=tf.keras.activations.relu)
        self.fc2 = tf.keras.layers.Dense(units=kargs['output_dimension'],
                               activation=tf.keras.activations.softmax)
        
    def call(self, x):
        x = self.embedding(x)
        #x = self.dropout(x)
        x = self.lstm_2_layer(x)
        #x = self.fc1(x)
        #x = self.dropout(x)
        x = self.fc2(x)
        
        return x     


<h3> 모델 함수 설정 (임베딩 적용)

In [30]:
"""n_hidden = 200
n_in = 32
model = Sequential()
model.add(Embedding(max_words,embedding_dim,input_length=maxlen))
model.add(LSTM(32,dropout=0.2))
model.add(Dense(n_hidden, input_dim=n_in))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(5, activation = 'softmax'))
"""#model.summary()

"n_hidden = 200\nn_in = 32\nmodel = Sequential()\nmodel.add(Embedding(max_words,embedding_dim,input_length=maxlen))\nmodel.add(LSTM(32,dropout=0.2))\nmodel.add(Dense(n_hidden, input_dim=n_in))\nmodel.add(Activation('relu'))\nmodel.add(Dropout(0.2))\n\nmodel.add(Dense(5, activation = 'softmax'))\n"

In [31]:
n_class = 5
def get_base_model(**kargs):
    #model = RNNClassifier(**kargs)

    model = Sequential([
        Embedding(max_words, embedding_dim, weights = [embedding_matrix], trainable =False ,input_length=maxlen),
        #Bidirectional(LSTM(24,dropout=0.2,return_sequences=True)),
        Bidirectional(LSTM(24,dropout=0.2)),
        Dense(n_class, activation='softmax')
    ])
    #model.layers[0].weights([embedding_matrix])
    #model.layers[0].trainable = False
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=.01),
              metrics=['accuracy'])
    
    return model

In [32]:
#demo = get_base_model(**kargs)

<h3>모델 훈련과 평가

In [33]:
"""model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())"""

"model.compile(loss='sparse_categorical_crossentropy',\n              optimizer='adam',\n              metrics=['accuracy'])\nprint(model.summary())"

In [34]:
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [35]:
padding_type='post'
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=maxlen)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=maxlen)

In [36]:
"""num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)"""

'num_epochs = 20\nhistory = model.fit(train_padded, y_train, \n                    epochs=num_epochs, verbose=2, \n                    validation_split=0.2)'

In [37]:
n_fold = 5
n_class = 5
seed = 777
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [38]:
train_padded.shape , test_padded.shape, y_train.shape

((181013, 1000), (19617, 1000), (181013,))

In [39]:

#s.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
#f.keras.backend.clear_session()

In [40]:
#demo.fit(train_padded,y_train,epochs=10)

In [41]:
from keras.utils import to_categorical 
trn = train_padded
tst = test_padded
y = y_train
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))

for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    
    clf = get_base_model(**kargs)
    
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5,
                       verbose=0, mode='min', baseline=None, restore_best_weights=True)


    clf.fit(trn[i_trn], 
            y[i_trn],
            validation_data=(trn[i_val], y[i_val]),
            epochs=20,
            batch_size = 64,
            callbacks=[es],
           verbose = 1)
    p_val[i_val, :] = clf.predict(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold
    del(clf)
    gc.collect()
    tf.keras.backend.clear_session()


training model for CV #1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GP

Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /j

Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /j

  73/2263 [..............................] - ETA: 1:45 - loss: 1.5189 - accuracy: 0.3330Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
  75/2263 [..............................] - ETA: 1:45 - loss: 1.5174 - accuracy: 0.3350Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
  78/2263 [>.............................] - ETA: 1:45 - loss: 1.5153 - accuracy: 0.3357Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
  80/2263 [>.............................] - ETA: 1:45 - loss: 1.5130 - accuracy: 0.3387Executing op __inference_train_function_5811 in 

Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 129/2263 [>.............................] - ETA: 1:43 - loss: 1.4933 - accuracy: 0.3554Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 131/2263 [>.............................] - ETA: 1:43 - loss: 1.4910 - accuracy: 0.3562Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 133/2263 [>.............................] - ETA: 1:43 - loss: 1.4897 - accuracy: 0.3571Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 136/2263 [>........................

...

Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 322/2263 [===>..........................] - ETA: 1:33 - loss: 1.1660 - accuracy: 0.5431Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 324/2263 [===>..........................] - ETA: 1:33 - loss: 1.1664 - accuracy: 0.5429Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 326/2263 [===>..........................] - ETA: 1:33 - loss: 1.1655 - accuracy: 0.5432Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 328/2263 [===>.....................

 378/2263 [====>.........................] - ETA: 1:30 - loss: 1.1654 - accuracy: 0.5443Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 380/2263 [====>.........................] - ETA: 1:30 - loss: 1.1657 - accuracy: 0.5441Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 382/2263 [====>.........................] - ETA: 1:30 - loss: 1.1662 - accuracy: 0.5438Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 386/2263 [====>.........................] - ETA: 1:30 - loss: 1.1662 - accuracy: 0.5442Executing op __inference_train_function_5811 in 

Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 434/2263 [====>.........................] - ETA: 1:28 - loss: 1.1651 - accuracy: 0.5436Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 436/2263 [====>.........................] - ETA: 1:27 - loss: 1.1652 - accuracy: 0.5436Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 438/2263 [====>.........................] - ETA: 1:27 - loss: 1.1654 - accuracy: 0.5434Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 440/2263 [====>....................

 489/2263 [=====>........................] - ETA: 1:25 - loss: 1.1680 - accuracy: 0.5418Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 491/2263 [=====>........................] - ETA: 1:25 - loss: 1.1678 - accuracy: 0.5419Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 493/2263 [=====>........................] - ETA: 1:25 - loss: 1.1681 - accuracy: 0.5418Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5811 in device /job:localhost/replica:0/task:0/device:GPU:0
 495/2263 [=====>........................] - ETA: 1:25 - loss: 1.1677 - accuracy: 0.5420Executing op __inference_train_function_5811 in 

InternalError:    Failed to call ThenRnnForward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 50, 24, 1, 1000, 64, 24] 
	 [[{{node CudnnRNN}}]]
	 [[sequential/bidirectional/forward_lstm/PartitionedCall]] [Op:__inference_train_function_5811]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
from sklearn.metrics import log_loss

In [ ]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']

val_acc  = history.history['val_accuracy']
val_loss = history.history['val_loss']
epochs = range(1,len(acc)+1)


plt.plot(epochs, acc , 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label= 'Validation loss')
plt.title('Training and validation loss')
plt.show()


<h2> Logistic Regression

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [ ]:
scores = cross_val_score(LogisticRegression(), X_train, y_train,cv=5)
print("cv test score : {:/2f}".format(np.mean(scores)))

In [ ]:
scores = cross_val_score(LogisticRegression(), X_train_origin, y_train_origin,cv=5)
print("cv test score : {:/2f}".format(np.mean(scores)))